In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, ToolCall
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import tools_condition, ToolNode

from dotenv import load_dotenv

load_dotenv()

True

In [21]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    max_tokens=None,
)

In [22]:
# Define tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers
    
    Args:
        a: first int
        b: second int
    """
    
    return a * b

def add(a: int, b:int) -> int:
    """Multiply two numbers
    
    Args:
        a: first int
        b: second int
    """
    
    return a + b

def divide(a: int, b:int) -> float:
    """Multiply two numbers
    
    Args:
        a: first int
        b: second int
    """
    
    return a / b

In [23]:
# bind tools to model
tools = [add, multiply, divide]
llm_with_tools = llm.bind_tools(tools)

In [24]:
llm_with_tools.invoke('add 5 + 20 and multiply 2 * 3').tool_calls

[{'name': 'add',
  'args': {'a': 5.0, 'b': 20.0},
  'id': 'a00bc45a-1f4c-445f-8404-5b34b4741bdc',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 2.0, 'b': 3.0},
  'id': '35fbd319-1b49-474c-a64c-17bab248a11d',
  'type': 'tool_call'}]

In [25]:
# Define state
class State(MessagesState):
    pass

In [26]:
# Node1
def assistent(state: State) -> State:
    message = state['messages']
    llm_response = llm_with_tools.invoke(message)
    return {'messages': [llm_response]}

In [27]:
llm_tools_json = llm_with_tools.kwargs['tools']
tools_by_name = {tool_dict['function']['name']: tool_func for tool_dict, tool_func  in zip(llm_tools_json, tools)}

In [28]:
# Tool Execution node
def tool_execution(state: State):
    for tool_call in state['messages'][-1].tool_calls:
        tool = tools_by_name[tool_call['name']]
        tool_args_dict = tool_call['args']
        tool_result = tool(**tool_args_dict)
    return {'messages' :[ToolMessage(content=str(tool_result), name=tool_call['name'], tool_call_id=tool_call['id'])]}

In [29]:
def tool_routing_condition(state: State):
    if state['messages'][-1].content:
        return 'end'
    else:
        return "continue"

In [30]:
graph = StateGraph(State)

graph.add_node('llm', assistent)
graph.add_node('tools', tool_execution)
graph.add_edge(START, 'llm')
graph.add_edge('tools', 'llm')
graph.add_conditional_edges('llm',
                            path=tool_routing_condition,
                            path_map={'continue': 'tools', 'end': END} )


graph = graph.compile()


In [31]:
output = graph.invoke({'messages':[HumanMessage(content='what is 2 * 5')]})

In [32]:
for m in output['messages']:
    print(m.pretty_print())

================================ Human Message =================================

what is 2 * 5
None
================================== Ai Message ==================================
Tool Calls:
  multiply (d5102f06-d928-4d4f-9a42-3f2fdb8ea16d)
 Call ID: d5102f06-d928-4d4f-9a42-3f2fdb8ea16d
  Args:
    a: 2.0
    b: 5.0
None
================================= Tool Message =================================
Name: multiply

10.0
None
================================== Ai Message ==================================

The answer is 10.
None
